In [3]:
from amplpy import AMPL, ampl_notebook
import pandas
import matplotlib.pyplot as plt
import numpy

In [5]:
license = "a08a4945-4807-4400-83da-e35bdca430e4"
ampl = ampl_notebook(
    modules=["highs", "cbc", "gurobi", "cplex"], # pick from over 20 modules including most commercial and open-source solvers
    license_uuid=license) # La licencia del curso. Si la cambia, esto no funciona.

Licensed to AMPL Academic Community Edition License for <j.palmaespinosa@uandresbello.edu>.


---
# Introducción

AMPL (A Mathematical Programming Language) es un lenguaje diseñado para expresar este tipo de problemas —con precisión matemática, pero en un formato legible y reutilizable—. No es un lenguaje de programación general. Es un lenguaje de modelado: permite escribir el “qué” del problema (el modelo), sin preocuparse por el “cómo” resolverlo.

Entonces, AMPL es un *lenguaje de modelado algebraico*, que permite formular y resolver problemas de optimización (LP, MILP, NLP, etc.). AMPL fue diseñado para ser legible y estructurado como una descripción matemática del problema.  Se usa junto a solvers como CPLEX, Gurobi, o MINOS.

Una de las grandes ventajas de AMPL es que permite una separación y claridad entre modelo, datos y resultado.  Además, es un lenguaje escalable desde prototipos a problemas reales.  Esto significa que con poco o ningún cambio en el modelo, es factible resolver una gran gantidad de problemas.

## Archivos principales
Para lograr ejecutar un modelo y obtener resultados claros y reproducibles, se suele separar el problema en tres áreas y, por tanto, en tres archivos.

- Modelo (.mod): describe las variables, restricciones y función objetivo.
- Datos (.dat): proporciona los valores concretos para los parámetros del modelo.
- Script (.run o consola): ejecuta el modelo y muestra resultados.

En este caso, es posible escribir directamente el modelo o cargarlo desde un archivo aparte.

**Ejemplo**
> Ud. tiene una panadería donde fabrica dos tipos de productos: pan integral y pan dulce.
Dispone de 300 kilos de harina y 180 horas de uso de horno. El pan integral usa 1.5 kg de harina, 2  hrs de horno y proporciona una ganancia de 5u.m por pieza.  Por otro lado, el pan dulce usa 2.0 kg de harina, una hora de horneado y le entrega una ganancia de 4 u.m.  Determine la cantidad de panes a hornear para maximizar su ganancia.  Para ello, *plantee el problema de Programación lineal.*

1. Escriba el modelo que permita resolver este ejemplo, **en papel**
2. Observe cómo es implementado en AMPL

---
# Ejemplo 1

In [37]:
%%writefile panaderia.mod
# Acá Ud. debe escribir su modelo.  
# La linea %%writefile nombre.mod guarda lo que ud. escriba
# como nombre.mod. Es lo primero que debe ir en este bloque
# Este será el modelo que usaremos para resolver el problema


######
# EJEMPLO 1
# Ud. puede nombrar a sus variables,
# restricciones y F.O. como lo desee.
######


# Variables.
# Observe además que se incorporan
# las restricciones de No-Negatividad.
var x >= 0; #pan integral
var y >= 0; #pan dulce

# Función objetivo.
maximize Z: 5*x + 4*y;

# Restricciones.
subject to harina: 1.5*x + 2*y <= 300;
s.t.  horno: 2*x + y <= 180;

Writing panaderia.mod


In [38]:
#Cargamos el entorno de ejecución AMPL
# Esto se hace sólo una vez
ampl = AMPL()

In [39]:
# Carga del modelo.
ampl.read("panaderia.mod")

In [40]:
# Solve with CPLEX
ampl.solve(solver="cplex")
#Esto es para cerciorarse que el sistema se resolvió
assert ampl.solve_result == "solved"

# Mostramos la respuesta.
for var in ["x","y","Z"]:
    ampl.display(var)

CPLEX 22.1.2: optimal solution; objective 648
2 simplex iterations
x = 24

y = 132

Z = 648



Observe que la ganancia máxima es de 648 u.m., elaborando 24 panes integrales y 132 panes dulces

---

# Ejemplo 2

Cómo pudo observar, la idea es simple:

```python
# 1. Cargar AMPL
ampl = AMPL()
# 2. Cargar el modelo dado un archivo MODELO.mod
ampl.read("MODELO.mod")
# 3. Resolver con algún solver (puede revisar la documentación de AMPL).
ampl.solve(solver="cplex")
```

Use la misma estructura para resolver el siguiente problema:


> Una empresa fabrica tres productos: A, B y C. Cada uno requiere tiempo de máquina, material y mano de obra, con las siguientes necesidades por unidad producida:


| Recurso           | Producto A | Producto B | Producto C | Disponibilidad |
|:-----------------:|:----------:|:----------:|:----------:|:--------------|
| Tiempo (h)        |          2 |          1 |          3 |            100 |
| Material (kg)     |          3 |          4 |          2 |            120 |
| Mano de obra (h)  |          1 |          2 |          1 |             90 |


> Además, cada producto genera una utilidad:
> - A: 30 unidades monetarias
> - B: 20 unidades monetarias
> - C: 40 unidades monetarias
>   
> Determine cuántas unidades de cada producto deben producirse para maximizar la utilidad total, respetando las restricciones de recursos.

In [36]:
# RESOLUCION EJEMPLO 2

---